In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from copy import deepcopy


Using TensorFlow backend.


In [2]:
x = np.array([[1],[2],[3],[4]])
y = np.array([[2],[4],[6],[8]])

In [3]:
model = Sequential()
model.add(Dense(3, activation='relu', input_shape=(1,)))
model.add(Dense(1, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [19]:
from keras.optimizers import sgd
s = sgd(lr = 0.01)
model.compile(optimizer=s,loss='mean_squared_error',metrics=['accuracy'])

display(model.weights)

[<tf.Variable 'dense_1/kernel:0' shape=(1, 3) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(3, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]

In [9]:
def feed_forward(inputs, outputs, weights):
    pre_hidden = np.dot(inputs,weights[0])+ weights[1]
    hidden = np.where(pre_hidden<0, 0, pre_hidden) 
    out = np.dot(hidden, weights[2]) + weights[3]
    squared_error = (np.square(out - outputs))
    return squared_error

In [10]:
def update_weights(inputs, outputs, weights, epochs):  
    for epoch in range(epochs):
        org_loss = feed_forward(inputs, outputs, weights)

        weights_new = deepcopy(weights)
        weights_new2 = deepcopy(weights)

        for i in range(len(weights)):
            print(i,len(weights[-(i+1)].flatten()))
            for j in range(len(weights[-(i+1)].flatten())):
                print(j)
                if i<3:
                    weights_new[-(i+1)][j] += 0.0001
                    print('weights_new:', weights_new)
                    loss = feed_forward(inputs, outputs, weights_new)
                    print('loss', loss)
                    del_loss = np.sum(org_loss - loss)/(0.0001*len(inputs))

                    weights_new2[-(i+1)][j] += del_loss*0.01

                    weights_new = deepcopy(weights)

                else:
                    weights_new[-(i+1)][0][j] += 0.0001
                    print('weights_new:', weights_new)
                    loss = feed_forward(inputs, outputs, weights_new)
                    print('loss', loss)
                    del_loss = np.sum(org_loss - loss)/(0.0001*len(inputs))

                    weights_new2[-(i+1)][0][j] += del_loss*0.01

                    weights_new = deepcopy(weights)

        weights = deepcopy(weights_new2)
        
    return weights_new2

In [12]:
def update_weights(inputs, outputs, weights, epochs):  
    for epoch in range(epochs):
        org_loss = feed_forward(inputs, outputs, weights)

        weights_new = deepcopy(weights)
        weights_new2 = deepcopy(weights)

        for i, layer in enumerate(reversed(weights)):
            # print(i, layer)
            for index, weight in np.ndenumerate(layer):
                # print(index, weight)
                weights_new[-(i+1)][index] += 0.0001
                # print('weights_new:', weights_new)
                loss = feed_forward(inputs, outputs, weights_new)
                # print('loss', loss)
                del_loss = np.sum(org_loss - loss)/(0.0001*len(inputs))

                weights_new2[-(i+1)][index] += del_loss*0.01
                weights_new = deepcopy(weights)

        weights = deepcopy(weights_new2)
        
    return weights_new2

w = deepcopy(model.get_weights())
update_weights(x,y,w,1)

[array([[-0.07957256, -0.5579188 , -0.88231397]], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([[-0.59802526],
        [-1.138372  ],
        [ 1.1945511 ]], dtype=float32),
 array([0.099999], dtype=float32)]

In [13]:
model.fit(x, y, epochs=1, batch_size=4, verbose=1)
model.get_weights()

Epoch 1/1
4/4 [==============================] - 0s 19ms/step - loss: 30.0000 - acc: 0.0000e+00


[array([[-0.07957256, -0.5579188 , -0.88231397]], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([[-0.59802526],
        [-1.138372  ],
        [ 1.1945511 ]], dtype=float32),
 array([0.09999999], dtype=float32)]

In [14]:
x1 = [[5],[6]]
model.predict(np.array(x1))

array([[0.09999999],
       [0.09999999]], dtype=float32)

In [15]:
def cat_cross_entropy(p, y):
    return -np.sum((y*np.log2(p)+(1-y)*np.log2(1-p)))

In [16]:
cat_cross_entropy(np.array([0.1,0.9]),np.array([0,1]))

0.3040061868900999

In [17]:
np.log2(0.9)

-0.15200309344504995